In [6]:
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import numpy as np
import pandas as pd

from free_utils import (
    file_names,
    data_folder
)

In [10]:
file_path = '../data_updated/' + file_names[0]

# Read the file into a DataFrame
dataset = pd.read_csv(file_path)
dataset_with_shift=dataset
#Сдвигаем на одну позицию вперед целевую переменную
#Target_1 это лаг наооборот для целевой переменной , типа предсказываем на день вперед
dataset_with_shift['Target_1'] = dataset['Цена'].shift(1)
dataset_with_shift.dropna(inplace=True)

dataset_with_shift

,Дата,Цена,Откр.,Макс.,Мин.,Объём,Изм. %,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,SMA_20,EMA_20,RSI,MACD,Signal_Line,Target_1
0,2013-01-14,2020.0,2010.9,2021.4,2001.9,117000000,0.69,0,1,1,2013,14,14,3,2006.070,2004.281941,51.827243,5.544697,5.470472,NaN
1,2013-01-15,2006.0,2018.3,2018.6,2000.2,55997000,-0.69,1,1,1,2013,15,15,3,2004.785,2002.627408,49.582754,4.699511,5.451916,2020.0
2,2013-01-16,2007.6,2006.5,2011.0,1990.5,70856000,0.08,2,1,1,2013,16,16,3,2003.990,2002.272398,47.187293,4.977845,5.640017,2006.0
3,2013-01-17,2012.7,2013.3,2020.0,1996.0,94260000,0.25,3,1,1,2013,17,17,3,2004.120,2001.711598,52.541296,5.097786,5.805560,2007.6
4,2013-01-18,2017.0,2016.0,2022.0,2012.4,65519000,0.21,4,1,1,2013,18,18,3,2003.735,2000.554924,55.273189,4.657452,5.982504,2012.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2689,2023-10-09,6863.5,6880.0,6895.5,6822.0,111000000,0.98,0,4,10,2023,282,9,41,NaN,7137.807524,NaN,-75.158655,-30.024858,6797.0
2690,2023-10-10,6853.5,6874.0,6880.0,6822.5,55220000,-0.15,1,4,10,2023,283,10,41,NaN,7166.682000,NaN,-58.707016,-18.741408,6863.5
2691,2023-10-11,6890.5,6864.5,7099.5,6835.5,347000000,0.54,2,4,10,2023,284,11,41,NaN,7199.648526,NaN,-35.836641,-8.750006,6853.5
2692,2023-10-12,7120.0,6862.5,7154.0,6848.0,272000000,3.33,3,4,10,2023,285,12,41,NaN,7232.190476,NaN,-9.891738,-1.978348,6890.5


In [11]:
X = dataset.drop(['Цена','Дата'],axis=1)
y = dataset['Target_1']  # Целевая переменная

In [ ]:


X = X[-len(X_ltsm):]
y = y[-len(X_ltsm):]


# Разделение данных на обучающую и тестовую выборки
X_train,X_test = X[:train_size], X[train_size:]
y_train,y_test = y[:train_size], y[train_size:]

In [ ]:

# Convert y_train and y_test to DataFrame objects
y_train_df = pd.DataFrame(y_train)
y_test_df = pd.DataFrame(y_test)

# Save X_train, X_test, y_train_df, and y_test_df to Parquet files
X_train.to_parquet('data/X_train.parquet')
X_test.to_parquet('data/X_test.parquet')
y_train_df.to_parquet('data/y_train.parquet')
y_test_df.to_parquet('data/y_test.parquet')

params = {
    'tree_method': 'hist',
    'device': 'cuda',
    'max_depth': 4,
    'n_estimators': 1100,
    'learning_rate': 0.2683229089980892,
    'subsample': 0.9,
    'gamma': 0.16553627228872347,
    'min_child_weight': 0.999734816446258,
    'max_delta_step': 0.32058180713235257,
    'colsample_bytree': 0.6664839820394654,
    'reg_alpha': 0.032217023806391465,
    'reg_lambda': 0.0010841335583071654,
    'scale_pos_weight': 0.2122327545777681
}

In [ ]:

# Создание и обучение модели XGBoost
# model_xgb = xgb.XGBRegressor(max_depth=3, n_estimators=50, learning_rate = 0.5, gamma=0.6, alpha=0.2, reg_lambda=0.2,tree_method='hist',device = 'cuda')  # Подберите параметры по необходимости
model_xgb = xgb.XGBRegressor(**params)  # Подберите параметры по необходимости
model_xgb.fit(X_train, y_train)

# Прогнозирование
xgb_predictions_train = model_xgb.predict(X_train)


# Create and train the Random Forest model
model_rf = RandomForestRegressor(max_depth=3, n_estimators=200,max_features= 'log2' )  # Adjust the parameters as needed
model_rf.fit(X_train, y_train)

# Predict
rf_predictions_train = model_rf.predict(X_train)

In [ ]:
xgb_predictions_test = model_xgb.predict(X_test)

# Рассчет метрик
mse = mean_squared_error(y_test, xgb_predictions_test)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, xgb_predictions_test)

print('RMSE (Root Mean Squared Error):', rmse)
print('r2 (Root Mean Squared Error):', r2)

rf_predictions_test = model_rf.predict(X_test)

# Calculate metrics
mse = mean_squared_error(y_test, rf_predictions_test)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, rf_predictions_test)

print('RMSE (Root Mean Squared Error):', rmse)
print('r2 (Root Mean Squared Error):', r2)

In [ ]:
xgb.plot_importance(model_xgb)
plt.show()